In [75]:
%pip install "git+https://github.com/ScierKnave/TorchMPS.git"
!git clone "https://github.com/ScierKnave/honor_project.git"
import requests
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
%pip install torchmetrics
from torchmetrics.classification import MulticlassAccuracy
from torchmps import MPS
import math

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ScierKnave/TorchMPS.git to /tmp/pip-req-build-o37b8yz5
  Running command git clone --filter=blob:none --quiet https://github.com/ScierKnave/TorchMPS.git /tmp/pip-req-build-o37b8yz5
  Resolved https://github.com/ScierKnave/TorchMPS.git to commit f716a08e15d0af50dbfdfc435ab9604e82562ea3
  Preparing metadata (setup.py) ... done
fatal: destination path 'honor_project' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Hyperparameters

In [76]:
# Hardware hyperparameters
chosen_device = torch.device('cuda' 
if torch.cuda.is_available() else 'cpu')

# Data hyperparameters
nb_train_HP = 2000
nb_test_HP = 500
batch_sz_HP = 150
batch_sz_HP = min(batch_sz_HP, nb_train_HP)
nb_classes_HP = 10

# Student hyperparameters
# MPS parameters
bond_dim_HP = 20
# USING CUSTOM FEATURE MAP WITH FORK?: YES

# Training parameters
nepochs_student_HP = 30 
student_lr_HP = 1e-4
student_reg_HP = 0
student_loss_HP = nn.CrossEntropyLoss()


# Premilinaries: Importing the data and utils subroutines

In [77]:

# Import the *common* mnist train set and create a batch iterator for it.
train_set = torch.load('/content/honor_project/src/datasets/train_mnist.pt')
train_iterator = torch.utils.data.DataLoader(
    dataset = train_set, 
    sampler = torch.utils.data.SubsetRandomSampler(range(nb_train_HP)),
    batch_size=batch_sz_HP
    )

# Import the mnist *common* test set and create a batch iterator for it.
test_set = torch.load('/content/honor_project/src/datasets/test_mnist.pt')
test_iterator = torch.utils.data.DataLoader(
    dataset = test_set, 
    sampler = torch.utils.data.SubsetRandomSampler(range(nb_test_HP)),
    batch_size = batch_sz_HP
    )

In [78]:
# Returns the validation set classification accuracy
# of the given input model (this is a higher order function)
def get_acc(model, iterator, dataset_size):
    # Get the validation set classification accuracy
    total_good_classifications = 0
    acc_metric = MulticlassAccuracy(num_classes=nb_classes_HP).to(chosen_device)
    for (x_mb, y_mb) in iterator:
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        y_mb = y_mb.to(chosen_device)
        # Add the number of datapoints we classified right to the total
        batch_size = x_mb.size()[0]
        y_hat = model(x_mb)
        batch_good_classifications = batch_size * acc_metric(y_hat, y_mb)
        total_good_classifications += batch_good_classifications
    return total_good_classifications / dataset_size # divide by total size

# Training the student model

In [84]:
def train_a_student():
  '''
  Trains a student model.
  '''

  # Initialize the MPS module
  student = MPS(
      input_dim = 28 ** 2,
      output_dim = 10,
      bond_dim = bond_dim_HP
  ).to(chosen_device) 
  #student.register_feature_map(feature_map_HP)

  # Instantiate the optimizer and softmax
  student_optimizer = torch.optim.Adam(
      student.parameters(), lr = student_lr_HP, weight_decay = student_reg_HP
  )

  # Used on the inputs before the loss function
  LogSoftmax = nn.LogSoftmax(dim=1)

  # Create an array to store the val loss
  # of the student at each epoch
  stud_test_loss = np.array([])
  stud_train_loss = np.array([])

  softmax = nn.Softmax(dim=1)

  stud_train_loss = np.array([])
  stud_test_loss = np.array([])

  for epoch in range(nepochs_student_HP):
      '''
      Training loop through the epochs.
      '''
      for (x_mb, y_mb) in train_iterator:
          # Flatten the MNIST images, which come in matrix form
          x_mb = x_mb.reshape(-1, 784).to(chosen_device)
          y_mb = y_mb.to(chosen_device)

          student_logits = student(x_mb) 

          # Backpropagation
          loss = student_loss_HP( student_logits, y_mb)

          student_optimizer.zero_grad()
          loss.backward()
          student_optimizer.step()

      # Get accuracy over all test and training data for current epoch
      train_current_accuracy = round( get_acc(student, train_iterator, nb_train_HP).item(), 4)
      test_current_accuracy = round( get_acc(student, test_iterator, nb_test_HP).item(), 4)
      stud_train_loss = np.append(stud_train_loss, train_current_accuracy)
      stud_test_loss = np.append(stud_test_loss, test_current_accuracy)
  return(stud_train_loss, stud_test_loss)


# Repeat the training process in order to get the variance
global_stud_test_loss = np.array([])
global_stud_train_loss = np.array([])
for i in range(5):
  print("\n Training and testing of model ", i+1, " in progress...")
  (stud_train_loss, stud_test_loss) = train_a_student()
  print("Train loss: ", stud_train_loss)
  print("Test loss: ", stud_test_loss)
  if (i == 0):
    global_stud_train_loss = stud_train_loss
    global_stud_test_loss = stud_test_loss
  else:
    global_stud_train_loss = np.vstack((global_stud_train_loss, stud_train_loss))
    global_stud_test_loss = np.vstack((global_stud_test_loss, stud_test_loss))




 Training and testing of model  1  in progress...
Train loss:  [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1
 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
Test loss:  [0.2 0.  0.  0.1 0.1 0.1 0.1 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0. ]

 Training and testing of model  2  in progress...
Train loss:  [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1
 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
Test loss:  [0.1 0.  0.  0.1 0.1 0.1 0.1 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0. ]

 Training and testing of model  3  in progress...
Train loss:  [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1
 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
Test loss:  [0.1 0.  0.  0.1 0.1 0.1 0.1 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0. ]

 Training and testing of model  4  in progress...
Train loss:  [0.1    0.1    0.1    0.2    0.2    0.2    0.1    0.1    0.1    0.1
 0.1    0.1    0.1    0.1

In [80]:
# Print the final results and save them for the report.
print("Final results")
print("Train loss: ", global_stud_train_loss)
print("Test loss: ", global_stud_test_loss)

np.save('mps_trainloss', stud_train_loss)
np.save('mps_testloss', stud_test_loss)


Final results
Train loss:  [[0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1
  0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1
  0.1    0.1    0.1    0.1    0.1   ]
 [0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1
  0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1
  0.1    0.1    0.1    0.1    0.1   ]
 [0.1    0.1    0.1    0.2    0.1    0.1    0.1    0.1    0.1    0.1
  0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1
  0.1    0.1    0.1    0.1    0.1   ]
 [0.1    0.1    0.1667 0.1    0.1    0.1    0.1    0.1    0.1    0.1
  0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1
  0.1    0.1    0.1    0.1    0.1   ]
 [0.1    0.1    0.1    0.1    0.1    0.1    0.2    0.1    0.1    0.1
  0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1
  0.1667 0.2    0.2    0.2    0.2   ]]
Test loss:  [[0.1  0.   0.   0.1  0.1  0.1  0.1  0.   0.   0.   0.   0.   0.   0.
  0.   0. 